# Notebook to calculate the voltgae drop between 200 and 240 minutes

## Load BDT files from folder and store as dataframes in a list

In [5]:
import os
import pandas as pd
import numpy as np

# Init main dir and list
main_directory = 'BDT_fleet_latest_Q4_2024'
fail_directory = 'BDT_fleet_failed'
latest_fleet_bdt = []
cb_terminal_voltage = ['RT1BMCBBV', 'RT4BMCBBV', 'RT5BMCBBV', 'RT8BMCBBV']
lb_terminal_voltage = ['RT1BMLBBV','RM2BMLBBV','RM3BMLBBV','RT4BMLBBV','RT5BMLBBV','RM6BMLBBV','RM7BMLBBV','RT8BMLBBV']

file_path = []

look_at_failed = False # Change depending on if you wanna look at pass or failed data

if look_at_failed:
    # Look through folder
    for file_name in os.listdir(fail_directory):
        if file_name.endswith('.csv'):
            temp_file_path = os.path.join(fail_directory, file_name)
            file_path.append(temp_file_path)
            file_path.sort()
else:       
    for file_name in os.listdir(main_directory):
        if file_name.endswith('.csv'):
            temp_file_path = os.path.join(main_directory, file_name)
            file_path.append(temp_file_path)
            file_path.sort()
        
# Read the CSV file into a df and put in list
for i, path in enumerate(file_path):
    df = pd.read_csv(file_path[i])
    latest_fleet_bdt.append(df)

# Check that files are loaded correctly
# and are sorted
print(file_path)

['BDT_fleet_latest_Q4_2024/B01_CSVLog-2024-04-23_2344.csv', 'BDT_fleet_latest_Q4_2024/B02_CSVLog-2024-10-16_0333.csv', 'BDT_fleet_latest_Q4_2024/B03_CSVLog-2024-09-09_0108.csv', 'BDT_fleet_latest_Q4_2024/B04_CSVLog-2024-09-10_1359.csv', 'BDT_fleet_latest_Q4_2024/B05_CSVLog-2024-08-14_0326.csv', 'BDT_fleet_latest_Q4_2024/B06_CSVLog-2024-07-31_0335.csv', 'BDT_fleet_latest_Q4_2024/B07_CSVLog-2024-02-29_2155.csv', 'BDT_fleet_latest_Q4_2024/B08_CSVLog-2024-08-26_0221.csv', 'BDT_fleet_latest_Q4_2024/B09_CSVLog-2024-09-18_0244.csv', 'BDT_fleet_latest_Q4_2024/B10_CSVLog-2024-09-27_0244.csv', 'BDT_fleet_latest_Q4_2024/B11_CSVLog-2024-10-07_0312.csv', 'BDT_fleet_latest_Q4_2024/B12_CSVLog-2024-10-24_2234.csv', 'BDT_fleet_latest_Q4_2024/B13_CSVLog-2024-10-22_2231.csv', 'BDT_fleet_latest_Q4_2024/B14_CSVLog-2024-06-19_0327.csv', 'BDT_fleet_latest_Q4_2024/B15_CSVLog-2024-11-03_0218.csv', 'BDT_fleet_latest_Q4_2024/B16_CSVLog-2024-10-29_2253.csv', 'BDT_fleet_latest_Q4_2024/B17_CSVLog-2024-05-23_0305.cs

## Determine different betwene the voltage at 240 minutes and 200 minutes. 
## For control battery only

In [6]:
D11_CB_200mins_voltage = []
T13_CB_200mins_voltage = []
T14_CB_200mins_voltage = []
D12_CB_200mins_voltage = []

D11_CB_terminal_voltage = []
T13_CB_terminal_voltage = []
T14_CB_terminal_voltage = []
D12_CB_terminal_voltage = []
cb_target = 12000
cb_tolerance = 30

temp_average_200 = 0
temp_average_240 = 0
drop = 0
drop_average = 0
drop_fleet_average = 0

# Loop through trains (B1-B41)
for i, train in enumerate(latest_fleet_bdt):
    temp_df = latest_fleet_bdt[i]
    temp_df_time = temp_df[temp_df.columns[0]]
    temp_df['datetime_column'] = pd.to_datetime(temp_df_time, format='mixed')
    temp_df['time_diff'] = (temp_df['datetime_column'] - temp_df['datetime_column'].iloc[0]).dt.total_seconds()
    filtered_df = temp_df[(temp_df['time_diff'] >= cb_target - cb_tolerance) & (temp_df['time_diff'] <= cb_target + cb_tolerance)]
    if not filtered_df.empty:
        # Extract final voltage value per CB per train
        D11_CB_terminal_voltage = latest_fleet_bdt[i][cb_terminal_voltage[0]].tail(1).values[0]
        T13_CB_terminal_voltage = latest_fleet_bdt[i][cb_terminal_voltage[1]].tail(1).values[0]
        T14_CB_terminal_voltage = latest_fleet_bdt[i][cb_terminal_voltage[2]].tail(1).values[0]
        D12_CB_terminal_voltage = latest_fleet_bdt[i][cb_terminal_voltage[3]].tail(1).values[0]
        
        # Extract 200m voltage value per CB per train
        D11_CB_200mins_voltage = filtered_df[cb_terminal_voltage[0]].iloc[0]
        T13_CB_200mins_voltage = filtered_df[cb_terminal_voltage[1]].iloc[0] 
        T14_CB_200mins_voltage = filtered_df[cb_terminal_voltage[2]].iloc[0] 
        D12_CB_200mins_voltage = filtered_df[cb_terminal_voltage[3]].iloc[0] 
        
        temp_average_200 = int(D11_CB_200mins_voltage+T13_CB_200mins_voltage+
                            T14_CB_200mins_voltage+D12_CB_200mins_voltage) / 4
        
        temp_average_240 = int(D11_CB_terminal_voltage+T13_CB_terminal_voltage+
                            T14_CB_terminal_voltage+D12_CB_terminal_voltage) / 4
        
        drop = temp_average_200-temp_average_240
        
        drop_average += drop
        
        print(f"200 mins average of train B{i+1} is {temp_average_200/10}V")
        print(f"240 mins average of train B{i+1} is {temp_average_240/10}V")
        print(f"40 mins average drop of train B{i+1} is {(drop)/10}V")
        print(" ")
    
    else:
        print(f"Not looking good for you, mate. Look at train B{i+1}")

num_trains = len(latest_fleet_bdt)
print(f"40 mins drop fleet average is: {(drop_average / num_trains)/10}")

    
    

200 mins average of train B1 is 97.3V
240 mins average of train B1 is 95.425V
40 mins average drop of train B1 is 1.875V
 
200 mins average of train B2 is 96.8V
240 mins average of train B2 is 94.75V
40 mins average drop of train B2 is 2.05V
 
200 mins average of train B3 is 97.875V
240 mins average of train B3 is 96.925V
40 mins average drop of train B3 is 0.95V
 
200 mins average of train B4 is 97.375V
240 mins average of train B4 is 96.4V
40 mins average drop of train B4 is 0.975V
 
200 mins average of train B5 is 97.125V
240 mins average of train B5 is 96.05V
40 mins average drop of train B5 is 1.075V
 
200 mins average of train B6 is 97.35V
240 mins average of train B6 is 96.475V
40 mins average drop of train B6 is 0.875V
 
200 mins average of train B7 is 98.025V
240 mins average of train B7 is 97.15V
40 mins average drop of train B7 is 0.875V
 
200 mins average of train B8 is 97.675V
240 mins average of train B8 is 96.625V
40 mins average drop of train B8 is 1.05V
 
200 mins aver

## Below Not Used

In [7]:
# D11_LB_200mins_voltage = []
# N17_LB_200mins_voltage = []
# N19_LB_200mins_voltage = []
# T13_LB_200mins_voltage = []
# T14_LB_200mins_voltage = []
# N18_LB_200mins_voltage = []
# N16_LB_200mins_voltage = []
# D12_LB_200mins_voltage = []

# D11_LB_terminal_voltage = []
# N17_LB_terminal_voltage = []
# N19_LB_terminal_voltage = []
# T13_LB_terminal_voltage = []
# T14_LB_terminal_voltage = []
# N18_LB_terminal_voltage = []
# N16_LB_terminal_voltage = []
# D12_LB_terminal_voltage = []

# lb_target = 12000
# lb_tolerance = 30

# temp_average_200 = 0
# temp_average_240 = 0
# drop = 0
# drop_average = 0
# drop_fleet_average = 0

# # Loop through trains (B1-B41)
# for i, train in enumerate(latest_fleet_bdt):
#     temp_df = latest_fleet_bdt[i]
#     temp_df_time = temp_df[temp_df.columns[0]]
#     temp_df['datetime_column'] = pd.to_datetime(temp_df_time, format='mixed')
#     temp_df['time_diff'] = (temp_df['datetime_column'] - temp_df['datetime_column'].iloc[0]).dt.total_seconds()
#     filtered_df = temp_df[(temp_df['time_diff'] >= cb_target - cb_tolerance) & (temp_df['time_diff'] <= cb_target + cb_tolerance)]
#     if not filtered_df.empty:
#         # Extract final voltage value per CB per train
#         D11_LB_terminal_voltage = latest_fleet_bdt[i][lb_terminal_voltage[0]].tail(1).values[0]
#         N17_LB_terminal_voltage = latest_fleet_bdt[i][lb_terminal_voltage[1]].tail(1).values[0]
#         N19_LB_terminal_voltage = latest_fleet_bdt[i][lb_terminal_voltage[2]].tail(1).values[0]
#         T13_LB_terminal_voltage = latest_fleet_bdt[i][lb_terminal_voltage[3]].tail(1).values[0]
#         T14_LB_terminal_voltage = latest_fleet_bdt[i][lb_terminal_voltage[4]].tail(1).values[0]
#         N18_LB_terminal_voltage = latest_fleet_bdt[i][lb_terminal_voltage[5]].tail(1).values[0]
#         N16_LB_terminal_voltage = latest_fleet_bdt[i][lb_terminal_voltage[6]].tail(1).values[0]
#         D12_LB_terminal_voltage = latest_fleet_bdt[i][lb_terminal_voltage[7]].tail(1).values[0]
        
#         # Extract 200m voltage value per CB per train
#         D11_LB_200mins_voltage = filtered_df[lb_terminal_voltage[0]].iloc[0]
#         N17_LB_200mins_voltage = filtered_df[lb_terminal_voltage[0]].iloc[0]
#         N19_LB_200mins_voltage = filtered_df[lb_terminal_voltage[0]].iloc[0]
#         T13_LB_200mins_voltage = filtered_df[lb_terminal_voltage[1]].iloc[0] 
#         T14_LB_200mins_voltage = filtered_df[lb_terminal_voltage[2]].iloc[0]
#         N18_LB_200mins_voltage = filtered_df[lb_terminal_voltage[0]].iloc[0]
#         N16_LB_200mins_voltage = filtered_df[lb_terminal_voltage[0]].iloc[0]
#         D12_LB_200mins_voltage = filtered_df[lb_terminal_voltage[3]].iloc[0] 
        
#         temp_average_200 = int(D11_CB_200mins_voltage+T13_CB_200mins_voltage+
#                             T14_CB_200mins_voltage+D12_CB_200mins_voltage) / 4
        
#         temp_average_240 = int(D11_CB_terminal_voltage+T13_CB_terminal_voltage+
#                             T14_CB_terminal_voltage+D12_CB_terminal_voltage) / 4
        
#         drop = temp_average_200-temp_average_240
        
#         drop_average += drop
        
#         print(f"200 mins average of train B{i+1} is {temp_average_200/10}V")
#         print(f"240 mins average of train B{i+1} is {temp_average_240/10}V")
#         print(f"40 mins average drop of train B{i+1} is {(drop)/10}V")
#         print(" ")
    
#     else:
#         print(f"Not looking good for you, mate Look at train B{i+1}")

# num_trains = len(latest_fleet_bdt)
# print(f"40 mins drop fleet avergae is: {(drop_average / num_trains)/10}")

    
    